In [ ]:
# Código empleado para la realización del trabajo de fin de máster titulado:

# Análisis de la evolución anual de la afluencia en el metro de Madrid

# Máster en Ciencia de Datos e Inteligencia de Negocios

# Universidad Complutense de Madrid

# Autores: Ángela Trueba Fernández y Manuel Méndez Hurtado

# Septiembre de 2025

In [ ]:
# Este código permite dibujar sobre el mapa de Madrid puntos de colores cuya intensidad indica la cantidad de 
# afluencia de pasajeros en la estación correspondiente. Se permite la selección de año y/o línea de la que
# se quiera representar la demanda.

In [6]:
import pandas as pd
import folium
import branca

def dibujar_mapa_metro(archivo_demanda, archivo_coords, año=None, linea=None, radio=7, output_html="mapa_metro.html"):
    """
    Genera un mapa de las estaciones del Metro de Madrid con la demanda de entradas y permite filtrar por año y línea.

    Parámetros:
        archivo_demanda (str): Ruta del archivo Excel con los datos de demanda histórica.
        archivo_coords (str): Ruta del archivo Excel con las coordenadas de las estaciones y líneas.
        año (int, opcional): Año específico a visualizar. Si es None, se suman todos los años.
        linea (str, opcional): Línea específica a visualizar (ejemplo: 'L3'). Si es None, se muestran todas las líneas.
        radio (int, opcional): Radio de los círculos en el mapa. Por defecto es 7.
        output_html (str): Nombre del archivo HTML donde se guardará el mapa.
    """
    # Cargar los archivos
    demanda_df = pd.read_excel(archivo_demanda)
    coords_df = pd.read_excel(archivo_coords)

    # Renombrar columnas para que coincidan
    demanda_df.rename(columns={"AÑO": "Año", "NOMBRE": "Nombre"}, inplace=True)

    # Limpiar espacios en nombres de columnas y valores
    demanda_df.columns = demanda_df.columns.str.strip()
    coords_df.columns = coords_df.columns.str.strip()
    demanda_df["Nombre"] = demanda_df["Nombre"].str.strip()
    coords_df["Nombre"] = coords_df["Nombre"].str.strip()

    # Filtrar por año si se especifica
    if año:
        demanda_df = demanda_df[demanda_df["Año"] == año]

    # Agrupar la demanda por estación y sumar las entradas
    demanda_total = demanda_df.groupby("Nombre", as_index=False)["Entradas"].sum()

    # Unir los DataFrames por el nombre de la estación
    df_final = demanda_total.merge(coords_df, on="Nombre", how="left")

    # Filtrar por línea si se especifica
    if linea:
        if linea not in coords_df.columns:
            print(f"Error: La línea '{linea}' no existe en el archivo de coordenadas.")
            return
        df_final = df_final[df_final[linea] == 1]

    # Eliminar filas con NA en Latitud o Longitud
    df_final = df_final.dropna(subset=["Latitud", "Longitud"])

    # Calcular el valor máximo de la columna 'Entradas'
    max_total = df_final["Entradas"].max()

    # Definir los rangos (feos) de colores
    rango1 = max_total / 4
    rango2 = max_total / 2
    rango3 = 3 * max_total / 4

    # Crear el mapa centrado en Madrid (en concreto, en Sol)
    madrid_map = folium.Map(location=[40.4168, -3.7038], zoom_start=12)

    # Crear un colormap basado en los valores de 'Entradas'
    colormap = branca.colormap.LinearColormap(
        colors=["blue", "green", "yellow", "red"],
        vmin=0, vmax=max_total
    ).to_step(index=[0, rango1, rango2, rango3, max_total])

    # Añadir los puntos al mapa
    for _, row in df_final.iterrows():
        if row["Entradas"] <= rango1:
            color = "blue"
        elif row["Entradas"] <= rango2:
            color = "green"
        elif row["Entradas"] <= rango3:
            color = "yellow"
        else:
            color = "red"

        folium.CircleMarker(
            location=[row["Latitud"], row["Longitud"]],
            radius=radio,  # Usamos el parámetro radio
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.6,
            popup=f"Nombre: {row['Nombre']}<br>Total: {row['Entradas']}"
        ).add_to(madrid_map)

    # Si se especifica un año, lo añadimos al mapa como texto
    if año:
        folium.Marker(
            location=[40.52, -3.45],  # Ajustar posición si es necesario
            icon=folium.DivIcon(
                html=f'<div style="font-size: 40px; font-weight: bold; color: black;">Año {año}</div>'
            )
        ).add_to(madrid_map)

    # Añadir la leyenda del colormap
    colormap.add_to(madrid_map)

    # Guardar el mapa como HTML
    madrid_map.save(output_html)
    print(f"Mapa generado: {output_html}")

# función con todos los datos
# dibujar_mapa_metro("Demanda Histórica MdM.xlsx", "DF_Generico.xlsx")

# función con una línea específica
# dibujar_mapa_metro("Demanda Histórica MdM.xlsx", "DF_Generico.xlsx", linea="L3")

# Probar la función con una línea y un año específico
dibujar_mapa_metro("Demanda Histórica MdM.xlsx", "DF_Generico.xlsx", año=2022, linea="L12")
dibujar_mapa_metro("Demanda Histórica MdM.xlsx", "DF_Generico.xlsx", año=2022)

Mapa generado: mapa_metro.html
Mapa generado: mapa_metro.html
